In [32]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
# from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.common.by import By
import time
import pandas as pd
from selenium.common.exceptions import NoSuchElementException
import re
import datetime
from tqdm.notebook import tqdm
import json

In [33]:
from selenium import webdriver

driver = webdriver.Chrome()
driver.implicitly_wait(5)

In [14]:
url ='https://www.adidas.com/us'
driver.get(url)
time.sleep(2)
# list_link = driver.find_elements(By.CSS_SELECTOR, 'li > a._flyouts_558sn_37 > li > a')
list_link = driver.find_elements(By.CSS_SELECTOR, 'div._header_container_1spbu_30 > div > ul > li > a')
list_product_link = []
for link in list_link:
        href = link.get_attribute("href")
        list_product_link.append(href)
list_product_link = list_product_link[:3]
list_product_link

['https://www.adidas.com/us/men',
 'https://www.adidas.com/us/women',
 'https://www.adidas.com/us/kids']

In [34]:
url ='https://www.nike.com/us'
driver.get(url)

In [11]:
list_product_link = ['https://www.nike.com/men','https://www.nike.com/women','https://www.nike.com/kids' ]
list_product_link

['https://www.nike.com/men',
 'https://www.nike.com/women',
 'https://www.nike.com/kids']

In [4]:
list_p_link = []

In [10]:
list_p_link = []
for i in list_product_link:
        url = i
        driver.get(url)
        list_link = driver.find_elements(By.CSS_SELECTOR, '.merchMenuWrapper > div > ul > li > a')
        for link in list_link:
                href = link.get_attribute("href")
                list_p_link.append(href)
len(list_p_link)

NameError: name 'list_product_link' is not defined

In [13]:
list_sp_link = []
for u in tqdm(list_p_link, desc='Loading data...'):
    try:
        driver.get(u)
        # Wait for the elements to be present on the page
        try:
            WebDriverWait(driver, 30).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'div.product-card > div > figure > a.product-card__link-overlay'))
            )
        except Exception as e:
            countinue  # Breaks the loop if elements are not found within the wait time

        list_link = driver.find_elements(By.CSS_SELECTOR, 'div.product-card > div > figure > a.product-card__link-overlay')
        if not list_link:
            continue  # Breaks the loop if no product links are found

        for link in list_link:
            href = link.get_attribute("href")
            list_sp_link.append(href)
    except Exception as e:
        continue

len(list_sp_link)

Loading data...:   0%|          | 0/48 [00:00<?, ?it/s]

1066

In [14]:
import pickle
pickle.dump(list_sp_link, open('list_nike_sp.pkl', 'wb'))

In [11]:
import pickle

# Path to your pickle file
pkl_file_path = 'C:/Users/nvatu/list_nike.pkl'

# Loading the data from the pickle file
with open(pkl_file_path, 'rb') as file:
    list_p_link = pickle.load(file)

# Now, loaded_list contains the data from your pickle file
len(list_p_link)

48

In [16]:
out_file = open('addidas_data.jsonl', 'a')

In [16]:
list_data = [] 

In [36]:
len(list_sp_link)

1066

In [18]:
wait_time = 2

for product_link in tqdm(list_sp_link, desc='Loading data...'):
    try:
        driver.get(product_link)
        now = datetime.datetime.now()
        scraped_at = now.strftime("%m/%d/%Y, %H:%M:%S")

        try:
            availability_text = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, '.pr1-sm'))).text
            availability = 'OutOfStock' if availability_text == 'Sold Out:' else 'InStock'
        except (TimeoutException, NoSuchElementException):
            availability = 'InStock'
        try:
            avg_rating_element = WebDriverWait(driver, wait_time).until(EC.presence_of_element_located((By.CSS_SELECTOR, '.css-rptnlm > h3:nth-child(1) > span')))
            avg_rating = avg_rating_element.text.strip()
        except (TimeoutException, NoSuchElementException):
            avg_rating = 0
        try:
            color = WebDriverWait(driver, wait_time).until(EC.presence_of_element_located((By.CSS_SELECTOR, '.single-color-label___29kFh > font > font'))).text
        except (TimeoutException, NoSuchElementException):
            try:
                color = WebDriverWait(driver, wait_time).until(EC.presence_of_element_located((By.CSS_SELECTOR, '.color-label___2hXaD'))).text
            except (TimeoutException, NoSuchElementException):
                try:
                    color = WebDriverWait(driver, wait_time).until(EC.presence_of_element_located((By.CSS_SELECTOR, '.single-color-label___29kFh'))).text
                except (TimeoutException, NoSuchElementException):
                    color = ''
    
        try:
            clickable_element = WebDriverWait(driver, wait_time).until(EC.presence_of_element_located((By.CSS_SELECTOR, '#navigation-target-description')))
            clickable_element.click()
            description = WebDriverWait(driver, wait_time).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div#navigation-target-description'))).text
        except (TimeoutException, NoSuchElementException):
            description = ''

        try:
            clickable_element = WebDriverWait(driver, wait_time).until(EC.presence_of_element_located((By.CSS_SELECTOR, '#navigation-target-specifications')))
            clickable_element.click()
            detail = WebDriverWait(driver, wait_time).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div#navigation-target-specifications'))).text
        except (TimeoutException, NoSuchElementException):
            detail = ''

        images = []
        try:
            image_elements = WebDriverWait(driver, wait_time).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'div > div > div > picture > img')))
            images = [img.get_attribute('src') for img in image_elements if img.get_attribute('src')]
        except (TimeoutException, NoSuchElementException):
            images = []
        try: 
            name = WebDriverWait(driver, wait_time).until(EC.presence_of_element_located((By.CSS_SELECTOR, '.sidebar___29cCJ > div:nth-child(1) > h1:nth-child(2) > span:nth-child(1)'))).text
        except (TimeoutException, NoSuchElementException):
            name = ''
        try:
            price_element = WebDriverWait(driver, wait_time).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div.sidebar___29cCJ > div.product-description___1TLpA > div.product-price___2Mip5.gl-vspace > div > div > div > div.gl-price-item.gl-price-item--sale.notranslate')))
            price = price_element.text.strip()
                
                # If sale price is found, also find the original price
            original_price_element = WebDriverWait(driver, wait_time).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div.product-price___2Mip5.gl-vspace > div > div > div > div.gl-price-item.gl-price-item--crossed.notranslate')))
            original_price = original_price_element.text.strip()
        except (TimeoutException, NoSuchElementException):
                # If sale price is not found, use the standard price for both price and original_price
            wait = WebDriverWait(driver, wait_time)  # Waits for wait_time seconds
            price_element = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.product-price___2Mip5.gl-vspace > div > div > div > div")))
            price = price_element.text.strip()
            original_price = price
        try:
            review_count_element = WebDriverWait(driver, wait_time).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div.accordion__header-text___2o9Im > div > h2')))
            review_count_text = review_count_element.text
            review_count_numbers = re.findall(r'\d+', review_count_text)
            review_count = int(review_count_numbers[0]) if review_count_numbers else 0
        except (TimeoutException, NoSuchElementException):
            review_count = 0

        category = WebDriverWait(driver, wait_time).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div.gallery-section___1kouT > ol > li:nth-child(4) > a > span'))).text 
        cloth_gender = WebDriverWait(driver, wait_time).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div.gallery-section___1kouT > ol > li:nth-child(3) > a > span'))).text
        data = {
            'availability': availability,
            'avg_rating': avg_rating,
            'brand': 'adidas',
            'color': color,
            'currency': 'USA',
            'description': description + '\n' + detail,
            'images': images,
            'name': name,
            'price': price,
            'original_price': original_price,
            'review_count': review_count, 
            'scraped_at': scraped_at,
            'url': product_link,
            'category': category,
            'cloth_gender': cloth_gender
        }
        list_data.append(data)
        # ghi xuống tạm thời chứ không nó mất
        out_file.write(json.dumps(data) + '\n')
    except Exception as e: 
        print(str(e))
        # continue

    # reset variables
    availability = None
    avg_rating = 0
    color = ''
    description = ''
    detail = ''
    images = []
    name = ''
    price = ''
    original_price = ''
    review_count = 0
    scraped_at = ''
    category = ''
    cloth_gender = ''
    
print("Len:", len(list_data))

Loading data...:   0%|          | 0/2585 [00:00<?, ?it/s]

Message: disconnected: received Inspector.detached event
  (Session info: chrome=120.0.6099.71)
Stacktrace:
	GetHandleVerifier [0x00007FF732B64D02+56194]
	(No symbol) [0x00007FF732AD04B2]
	(No symbol) [0x00007FF7329776AA]
	(No symbol) [0x00007FF732961D04]
	(No symbol) [0x00007FF732961723]
	(No symbol) [0x00007FF732960746]
	(No symbol) [0x00007FF7329606EB]
	(No symbol) [0x00007FF73295EE3D]
	(No symbol) [0x00007FF73295F7CE]
	(No symbol) [0x00007FF73297AF81]
	(No symbol) [0x00007FF73297BA98]
	(No symbol) [0x00007FF73297F38F]
	(No symbol) [0x00007FF73297F480]
	(No symbol) [0x00007FF7329C1494]
	(No symbol) [0x00007FF7329C17EC]
	(No symbol) [0x00007FF732A04D77]
	(No symbol) [0x00007FF7329E5EBF]
	(No symbol) [0x00007FF732A02786]
	(No symbol) [0x00007FF7329E5C23]
	(No symbol) [0x00007FF7329B4A45]
	(No symbol) [0x00007FF7329B5AD4]
	GetHandleVerifier [0x00007FF732EDD5BB+3695675]
	GetHandleVerifier [0x00007FF732F36197+4059159]
	GetHandleVerifier [0x00007FF732F2DF63+4025827]
	GetHandleVerifier [0x

In [19]:
out_file.close()

In [23]:
import json

with open('addidas_data.jsonl', 'r') as f:
    addidas_data = [json.loads(i) for i in f]

len(addidas_data)

2559